In [45]:
import pandas as pd
import numpy as np
import os

import sys

# Path to library directory.
_lib_path = os.path.join('c:' + os.sep, 'Users', 'natha', 'OneDrive', 'Documents', 'GitHub', 'Debiasing_Kepler_Data', 'lib')

# Allows access to modules in the library directory.
if _lib_path not in sys.path:
    sys.path.append(_lib_path)

import system_names

In [46]:
# .csv files:
# Masses in solar masses
# Radii in solar radii
# Semi-major axes in AU
# Use Kepler's 3rd for periods.

G = 6.6743 * 1e-11 # N⋅m 2 /kg 2
c = 3.00 * 1e8 # m 2 / s 2

SECONDS_PER_DAY = 24.0 * 60.0 * 60.0
METERS_PER_SOLAR_RADIUS = 6.955 * 1e8
KG_PER_SOLAR_MASS = 1.989 * 1e30
METERS_PER_AU = 149597870691.0
SECONDS_PER_YEAR = 31536000

def f_2(e):
    return (1.0 - e**2.0)**-5.0 * (1.0 + (3.0/2.0)*e**2.0 + (1.0/8.0)*e **4.0)

def apsidal_precession_tidal(k_2s, k_2p, R_s, R_p, M_s, M_p, a, e, n):
    return (15.0/2.0) * k_2s * (R_s/a)**5.0 * (M_p/M_s) * f_2(e) * n + \
            (15.0/2.0) * k_2p * (R_p/a)**5.0 * (M_s/M_p) * f_2(e) * n

def g_2(e):
    return (1.0 - e**2.0)**-2.0

def apsidal_precession_rotational(k_2s, k_2p, R_s, R_p, M_s, M_p, nu_s, nu_p, a, e, n):
    return (1.0/2.0) * k_2s * (R_s/a)**5.0 * (nu_s**2.0*a**3.0)/(G*M_s) * g_2(e) * n + \
            (1.0/2.0) * k_2p * (R_p/a)**5.0 * (nu_p**2.0*a**3.0)/(G*M_p) * g_2(e) * n

def apsidal_precession_general_relativity(M_s, a, e, n):
    return (2.0*G*M_s*n)/(c**2.0*a*(1.0 - e)**2.0)

def apsidal_precession_total(k_2s, k_2p, R_s, R_p, M_s, M_p, nu_s, nu_p, a, e, n):
    return apsidal_precession_tidal(k_2s=k_2s, k_2p=k_2p, R_s=R_s, R_p=R_p, M_s=M_s, M_p=M_p, a=a, e=e, n=n) + \
            apsidal_precession_rotational(k_2s=k_2s, k_2p=k_2p, R_s=R_s, R_p=R_p, M_s=M_s, M_p=M_p, nu_s=nu_s, nu_p=nu_p, a=a, e=e, n=n) + \
            apsidal_precession_general_relativity(M_s=M_s, a=a, e=e, n=n)

def generate_apsidal_precessions(savepath, system_filepaths):
    directory = os.path.join(os.pardir, os.pardir, 'data', 'processed', 'apsidal_precessions')
    catalog_filepath = os.path.join(directory, 'physical_catalog.csv')
    catalog_ids_filepath = os.path.join(directory, 'used_systems.csv')

    # Determines how many lines to read in.
    num_systems = 1000
    planet_multiplier = 6

    catalog_ids = pd.read_csv(catalog_ids_filepath)
    catalog = pd.read_csv(catalog_filepath, nrows = int(num_systems * planet_multiplier))

    save_file = open(savepath, 'w', encoding = 'utf-8')

    header = 'System Name,Apsidal Precession Period (Years)\n'

    save_file.write(header)

    for system_filepath in system_filepaths:
        system_name = system_filepath.split(os.sep)[-1].replace('.csv', '')
        system_number = int(system_name.split('_')[1])

        data = pd.read_csv(system_filepath)

        planet_mass_columns = [column for column in data.columns if ('Planet' in column and 'mass' in column)]
        planet_eccentricity_columns = [column for column in data.columns if ('Planet' in column and 'ecc' in column)]
        planet_radius_columns = [column for column in data.columns if ('Planet' in column and 'radius' in column)]
        planet_semi_major_axis_columns = [column for column in data.columns if ('Planet' in column and 'semi maj' in column)]

        # Gaurds against index errors.
        assert len(planet_mass_columns) == len(planet_eccentricity_columns) == len(planet_radius_columns) == \
                len(planet_semi_major_axis_columns)

        multiplicity = len(planet_mass_columns)

        k_2s = 1.0
        k_2p = 1.0

        M_s = data['Stellar mass'].iloc[0] * KG_PER_SOLAR_MASS
        R_s = data['Stellar radius'].iloc[0] * METERS_PER_SOLAR_RADIUS

        for index in range(multiplicity):
            star_id = catalog_ids['star_id'].iloc[system_number]

            # Massses should be in Solar masses for planets!
            # Radii should be in Solar radii for planets!
            # Massses should be in Solar masses for stars!
            # Radii should be in Solar radii for stars!

            # CHECK THAT THE PLANETS GET INDEXED IN ORDER FROM TOP TO BOTTOM FOR THE .csv LINE ENTRIES!!!
            T_catalog = catalog[catalog['star_id'] == star_id]['period'].iloc[index] * SECONDS_PER_DAY # Seconds.

            M_p = data[planet_mass_columns[index]].iloc[0] * KG_PER_SOLAR_MASS # kg.
            e = data[planet_eccentricity_columns[index]].iloc[0] # Unitless.
            R_p = data[planet_radius_columns[index]].iloc[0] * METERS_PER_SOLAR_RADIUS # Meters.
            a = data[planet_semi_major_axis_columns[index]].iloc[0] * METERS_PER_AU # Meters. 

            T = np.sqrt((4.0*np.pi**2.0)/(G*M_s) * a**3.0)

            n = (2.0*np.pi)/T # 1/seconds.

            # Ignores rotational effects for now.
            nu_s = 1/(10.0 * SECONDS_PER_DAY) # Spin period should be 10 days. Units in seconds.
            nu_p = 1/(10.0 * 60.0 * 60.0) # Spin period should be 10 hours. Units in seconds.

            # Converts to days.
            apsidal_precession_period = 1.0 / (SECONDS_PER_YEAR * apsidal_precession_total(k_2s=k_2s, k_2p=k_2p, R_s=R_s, R_p=R_p, M_s=M_s, M_p=M_p, nu_s=nu_s, nu_p=nu_p, a=a, e=e, n=n))

            line_output = system_name + ',' + str(apsidal_precession_period) + '\n'

            save_file.write(line_output)

    save_file.close()


In [48]:
path = os.path.join(os.pardir, os.pardir, 'data', 'raw')
file_names = os.listdir(path)

for unstable_system in system_names.unstable_systems:
    if unstable_system in file_names:
        file_names.remove(unstable_system)

filepaths = [os.path.join(path, filename) for filename in file_names]

save_path = os.path.join(os.pardir, os.pardir, 'data', 'processed', 'apsidal_precessions', 'apsidal_precessions.csv')

generate_apsidal_precessions(save_path, filepaths)